# Laboratoire 2
## Gyroscopes à taux (rate gyroscope)

**Matériel nécessaire fourni** : table tournante, surface antidérapante,
règle de 30 cm, masking tape, LEGOs inclinés.

Ce laboratoire utilise le gyro embarqué du robot Kobuki. Il s'agit du STMicroelectronics [L3G4200D](http://www.st.com/content/ccc/resource/technical/document/datasheet/04/46/d6/00/be/d9/46/ae/CD00265057.pdf/files/CD00265057.pdf/jcr:content/translations/en.CD00265057.pdf).

![img](img/gyro_schema.png)
![img](img/gyro_datasheet.png)

### Partie 1 - Familiarisation avec le capteur

In [ ]:
# Importation des modules

%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from robmob.robot import Robot
from robmob.sensors import GyroSensor
from robmob.visualization import Visualizer

Modifiez la valeur de `ip_robot` selon le robot qui vous a été assigné.

In [ ]:
# Connexion au robot
ip_robot = '192.168.0.100'
robot = Robot(ip_robot)
robot.connect()

Il faut ensuite ajouter le gyroscope aux capteurs écoutés par le robot.

In [ ]:
gyro = GyroSensor()
robot.add_sensor(gyro)

En guise de *sanity check*, le code suivant affiche la dernière mesure du gyro. Assurez-vous que le robot est immobile avant de prendre la mesure.

In [ ]:
gyro.peek_data()

Vous devriez remarquer qu'il y a passablement de bruit dans le capteur. Maintenant, installer le robot sur un plateau rotatif et faites le tourner à différentes vitesses. Remarquez le comportement de la mesure en z. Comment se comporte-t-elle à des vitesses de rotation plus élevées?

Le code suivant affiche la mesure en temps réel. Appuyer deux fois sur la touche *i* ou appuyez sur le stop dans le haut de la page pour arrêter la boucle.

In [ ]:
import time
from IPython.display import clear_output

while True:
    mesure = gyro.peek_data()
    clear_output(wait=True)
    print("x: %0.10f deg/sec" % mesure['x'])
    print("y: %0.10f deg/sec" % mesure['y'])
    print("z: %0.10f deg/sec" % mesure['z'])
    time.sleep(0.05)

> **NOTE** Selon la [documentation](http://kobuki.yujinrobot.com/wiki/gyro-details/) de la plateforme Kobuki, le gyroscope est utilisé en mode 250 dps.

### Partie 2 - Calibration du gyroscope
Selon la datasheet, la valeur de DVoff, ou *digital zero-rate level*, se situe entre -10 et 10 dps. Autrement dit, lorsque la plateforme est immobile, la mesure ne sera pas zéro, mais se situera entre -10 et +10 degrés par seconde.


Pour cette partie, laissez le robot immobile durant 30 secondes.
>**Attention!** Accrocher, faire vibrer (ou même souffler!) sur le capteur faussera le résultat.


In [ ]:
samples = gyro.sample_data_for_x_sec(30) # samples a 3 colonnes: x, y et z
z_samples = samples[:, 2]                # notation numpy pour toutes les lignes, colonne 2
print("Dimension de z_samples: ", z_samples.shape)

### Partie 3 - Intégration des mesures

### Partie 4 - Influence de l'angle du gyroscope

### Partie 5 -  Estimation de la dérive du gyroscope

### Partie 6 – Création d’une carte de l’environnement